In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import subprocess, os, sys

sys.path.insert(0, r'C:\Users\100700375\Documents\py_projects\MAD\src')
from data.setup import Setup
from data.mad import MAD

In [4]:
#carico i parametri di setup
setup= Setup()
#carico in locale i parametri di query e arricchimento dati
min_date=setup.Parameters.min_date #data da cui iniziare a leggere i dati
plant_in=setup.Parameters.plant_in # "(1,3,4) vcp, ccp,cfp
status_samord_min=setup.Parameters.status_samord_min #da attesa produzione in poi
status_cusreq_min=setup.Parameters.status_cusreq_min #da attesa valutazione in poi
days_on_time=int(setup.Parameters.days_on_time) #numero di giorni <=per cui ttm e otd sono on time

In [ ]:
RetriveCusReq_query=f"""SELECT 
    SAMREQ.IDSamReq as CusReq,
    CLIENTI.Cliente as CustomerName,
    SAMREQ.TitoloReq as Title,
    STATO.DesStato_ITA as StatusDesc,
    SAMREQ.StatoReq as Status,
    IF(SAMREQ.MADType=1,'STD','DEV') AS ReqType,
    IF(SAMREQ.MADType=1,1,0) as IsStdNr,
    IF(SAMREQ.MADType<>1,1,0) as IsDevNr,
    MAD.MADDesc as MAD,
    SUBSTRING(Plant.Plant,1,3) as PlantName,
    SUBSTRING(BU.BusinesseUnit,1,2) as BU,
    IF(SAMREQ.IsZZ=2,'Y','N') as IsZZ,    #2=ZZ, 3=Non ZZ
    CASE SAMREQ.IsNewPrd
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'New'
        WHEN 3 THEN 'SAP'
    END as IsNewPrd,
    CASE SAMREQ.IsConverted
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Converted'
        WHEN 3 THEN 'Foam'
    END as Conv_Foam,
    CASE SAMREQ.IsGreenList
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as InGreenList,
    CASE SAMREQ.IsStdMod
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Std_RM_Proc'
        WHEN 3 THEN 'New_RM_Proc'
    END as StdNew,
    CASE SAMREQ.IsBasic
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Basic'
        WHEN 3 THEN 'Custom'
    END as Specification,
    CASE SAMREQ.IsNewBU
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'New'
        WHEN 3 THEN 'Existing'
    END as NewBusiness,
    CASE SAMREQ.IsSample
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as SampleRequired,
    CASE SAMREQ.IsCost
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as CostRequired,
    SAMREQ.QtyYear*SAMREQ.PriceTgt as TO_TGT,
    SAMREQ.QtyYear*SAMREQ.PriceCur as TO_ACT,
    SAMREQ.GMTgt as MRG_TGT,
    SAMREQ.GMCur as MRG_ACT,
    CASE SAMREQ.IsSampleApproved
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as SampleApproved,
    CASE SAMREQ.IsRequestApproved
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as RequestApproved,
    SAMREQ.DateIns,SAMREQ.DateAssessment,SAMREQ.DateMod,SAMREQ.DateSalesBegin,
    EMP2.EmployeeName as EmployeeReq,EMP.EmployeeName as EvalMngName,
    COUNT(SAPCODE.SapCod) as SAPCodCount, MIN(SAPCODE.modtime) as SAPCodFirst,MAX(SAPCODE.modtime) as SAPCodLast
    FROM tblsamrequests as SAMREQ 
    INNER JOIN tlkpplant as Plant on SAMREQ.IDPlantReq= Plant.IDPlant 
    INNER JOIN tlkpbu as BU ON SAMREQ.IDBU=BU.IDBU 
    INNER JOIN ztblsamreqsts as STATO ON SAMREQ.StatoReq=STATO.StatoReq 
    INNER JOIN tblclienti as CLIENTI ON SAMREQ.IDCliente=CLIENTI.IDCliente 
    LEFT JOIN tlkpmad as MAD ON SAMREQ.IDMAD=MAD.IDMAD
    LEFT JOIN tblemployees as EMP ON SAMREQ.IDEmployeeEvalMngr =EMP.EmployeeID 
    LEFT JOIN tblemployees as EMP2 ON SAMREQ.IDEmpIns =EMP2.EmployeeID
    LEFT JOIN tblreqzzsapcode as SAPCODE ON SAMREQ.IDSamReq=SAPCODE.IDSamReq
    WHERE SAMREQ.DateIns>='{min_date}' AND Plant.IDPlant IN {plant_in} AND STATO.StatoReq >{status_cusreq_min}
    GROUP BY SAMREQ.IDSamReq;
    """

In [ ]:
RetriveCusReq_query_ori=f"""SELECT 
    SAMREQ.IDSamReq as CusReq,
    CLIENTI.Cliente as CustomerName,
    SAMREQ.TitoloReq as Title,
    STATO.DesStato_ITA as StatusDesc,
    SAMREQ.StatoReq as Status,
    IF(SAMREQ.MADType=1,'STD','DEV') AS ReqType,
    IF(SAMREQ.MADType=1,1,0) as IsStdNr,
    IF(SAMREQ.MADType<>1,1,0) as IsDevNr,
    MAD.MADDesc as MAD,
    SUBSTRING(Plant.Plant,1,3) as PlantName,
    SUBSTRING(BU.BusinesseUnit,1,2) as BU,
    IF(SAMREQ.IsZZ=2,'Y','N') as IsZZ,    #2=ZZ, 3=Non ZZ
    CASE SAMREQ.IsNewPrd
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'New'
        WHEN 3 THEN 'SAP'
    END as IsNewPrd,
    CASE SAMREQ.IsConverted
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Converted'
        WHEN 3 THEN 'Foam'
    END as Conv_Foam,
    CASE SAMREQ.IsGreenList
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as InGreenList,
    CASE SAMREQ.IsStdMod
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Std_RM_Proc'
        WHEN 3 THEN 'New_RM_Proc'
    END as StdNew,
    CASE SAMREQ.IsBasic
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Basic'
        WHEN 3 THEN 'Custom'
    END as Specification,
    CASE SAMREQ.IsNewBU
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'New'
        WHEN 3 THEN 'Existing'
    END as NewBusiness,
    CASE SAMREQ.IsSample
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as SampleRequired,
    CASE SAMREQ.IsCost
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as CostRequired,
    SAMREQ.QtyYear*SAMREQ.PriceTgt as TO_TGT,
    SAMREQ.QtyYear*SAMREQ.PriceCur as TO_ACT,
    SAMREQ.GMTgt as MRG_TGT,
    SAMREQ.GMCur as MRG_ACT,
    CASE SAMREQ.IsSampleApproved
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as SampleApproved,
    CASE SAMREQ.IsRequestApproved
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Y'
        WHEN 3 THEN 'N'
    END as RequestApproved,
    SAMREQ.DateIns,SAMREQ.DateAssessment,SAMREQ.DateMod,SAMREQ.DateSalesBegin,
    EMP2.EmployeeName as EmployeeReq,EMP.EmployeeName as EvalMngName
    FROM tblsamrequests as SAMREQ 
    INNER JOIN tlkpplant as Plant on SAMREQ.IDPlantReq= Plant.IDPlant 
    INNER JOIN tlkpbu as BU ON SAMREQ.IDBU=BU.IDBU 
    INNER JOIN ztblsamreqsts as STATO ON SAMREQ.StatoReq=STATO.StatoReq 
    INNER JOIN tblclienti as CLIENTI ON SAMREQ.IDCliente=CLIENTI.IDCliente 
    LEFT JOIN tlkpmad as MAD ON SAMREQ.IDMAD=MAD.IDMAD
    LEFT JOIN tblemployees as EMP ON SAMREQ.IDEmployeeEvalMngr =EMP.EmployeeID 
    LEFT JOIN tblemployees as EMP2 ON SAMREQ.IDEmpIns =EMP2.EmployeeID
    WHERE SAMREQ.DateIns>='{min_date}' AND Plant.IDPlant IN {plant_in} AND STATO.StatoReq >{status_cusreq_min}
    ;
    """

In [29]:
RetriveSapCod_query=f"""
SELECT 
SAMREQ.IDSamReq as CusReq,
CLIENTI.Cliente as CustomerName,
SAMREQ.TitoloReq as Title,
STATO.DesStato_ITA as StatusDesc,
IF(SAMREQ.MADType=1,'STD','DEV') AS ReqType,
CASE SAMREQ.IsNewPrd
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'New'
        WHEN 3 THEN 'SAP'
END as IsNewPrd,
CASE SAMREQ.IsConverted
        WHEN 1 THEN 'NA'
        WHEN 2 THEN 'Converted'
        WHEN 3 THEN 'Foam'
END as Conv_Foam,
MAD.MADDesc as MAD,
SUBSTRING(Plant.Plant,1,3) as PlantName,
SUBSTRING(BU.BusinesseUnit,1,2) as BU, 
IF(SAPCODE.IsZZ=-1,'Y','N') as IsZZ,    #-1=ZZ, 0=Non ZZ
SAPCODE.SapCod as SAPCod, SAPCODE.SapDes as SapDes, 
SAPCODE.modtime as DateMod, USERS.EmployeeName
FROM tblreqzzsapcode as SAPCODE 
INNER JOIN tblsamrequests as SAMREQ ON SAMREQ.IDSamReq=SAPCODE.IDSamReq 
INNER JOIN tlkpplant as Plant on SAMREQ.IDPlantReq= Plant.IDPlant 
INNER JOIN tlkpbu as BU ON SAMREQ.IDBU=BU.IDBU 
INNER JOIN ztblsamreqsts as STATO ON SAMREQ.StatoReq=STATO.StatoReq 
INNER JOIN tblclienti as CLIENTI ON SAMREQ.IDCliente=CLIENTI.IDCliente 
LEFT JOIN tblemployees as USERS ON SAPCODE.IDEmpIns=USERS.EmployeeID
LEFT JOIN tlkpmad as MAD ON SAMREQ.IDMAD=MAD.IDMAD
WHERE SAMREQ.DateIns>='{min_date}' AND Plant.IDPlant IN {plant_in} AND STATO.StatoReq >{status_cusreq_min};
"""

In [7]:
#creo l'oggetto MAD per la connessione al db
db = MAD(setup.Credentials)
#apro la connessione al db
db.open_connection()

Connected to MySQL Server version 8.0.41-0ubuntu0.22.04.1
You're connected to database: ('cm_db',)


In [30]:
db.execute_query(RetriveSapCod_query) #[['CusReq','Count_SAPCod','FirstInsert']]

Retrivied data from...........:  cm_db     


,CusReq,CustomerName,Title,StatusDesc,ReqType,IsNewPrd,Conv_Foam,MAD,PlantName,BU,IsZZ,SAPCod,SapDes,DateMod,EmployeeName
0,23-00011,JAEGGI INDUSTIES SRL,pls create Tr.Italy's SAP codes on the base of...,Chiuso,DEV,New,Converted,Modification,CCP,A,N,85006781,None,2023-01-23 11:31:08,Paleni S.
1,23-00011,JAEGGI INDUSTIES SRL,pls create Tr.Italy's SAP codes on the base of...,Chiuso,DEV,New,Converted,Modification,CCP,A,N,85009591,None,2023-01-23 11:31:03,Paleni S.
2,23-00011,JAEGGI INDUSTIES SRL,pls create Tr.Italy's SAP codes on the base of...,Chiuso,DEV,New,Converted,Modification,CCP,A,N,85291111,None,2023-01-23 11:31:05,Paleni S.
3,23-00012,WELLHÖFER TREPPEN GMBH,Generazione nuovo articolo 22-01354,Chiuso,DEV,New,Converted,Modification,CCP,A,N,85290801,None,2023-01-11 08:49:14,Paleni S.
4,23-00027,COZZI,FR C30 EN5NF08 K32,"Campione Consegnato, attesa feedback",STD,New,Foam,Modification,CFP,A,N,85291021,None,2023-01-18 08:59:42,Paleni S.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,23-00549,LIMONTA SPORT ITALIA,Modifica arrotolatura,Chiuso,DEV,New,Converted,Modification,CCP,E,N,85293231,None,2023-05-08 13:04:44,Simone Cattaneo
218,23-00549,LIMONTA SPORT ITALIA,Modifica arrotolatura,Chiuso,DEV,New,Converted,Modification,CCP,E,N,85293241,None,2023-05-08 13:04:54,Simone Cattaneo
219,24-01762,TROCELLEN ITALIA,Rimozione micro fori PMX1 da schiume tatami 35...,Chiuso,DEV,New,Foam,Modification,CFP,Pr,N,85212261,None,2024-11-06 13:03:32,M. Solagni
220,24-01762,TROCELLEN ITALIA,Rimozione micro fori PMX1 da schiume tatami 35...,Chiuso,DEV,New,Foam,Modification,CFP,Pr,N,85247551,None,2024-11-06 13:03:35,M. Solagni
